# globals



In [3]:
# lib
import os
import mediapipe as mp
import cv2
import numpy as np

# utils
from utils import actions,arg_max

# models & realTime
from pytorch_model import PytorchPredictor
from keras_model import KerasPredictor
from MultiSignDetector import MultiSignPredictor
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

n_classes = len(actions)

WEIGHTS_PATH=os.path.join("..","V1")
KERAS_WEIGHTS_PATH = os.path.join(WEIGHTS_PATH,"keras_weights","V1.h5")
TORCH_WEIGHTS_PATH = os.path.join(WEIGHTS_PATH,"pytorch_weights.tar")

# pytorch Model

In [4]:
pytorch_predictor = PytorchPredictor(path=TORCH_WEIGHTS_PATH)

# Keras model

In [5]:
keras_predictor = KerasPredictor(path=KERAS_WEIGHTS_PATH)

No GPU found
Num GPUs Available:  0


2022-03-10 19:17:38.447745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 19:17:38.448787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-10 19:17:38.449014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-10 19:17:38.449186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.

# MultiSign Detector

In [6]:
holistic = mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
fsize = (512,512)

multi_sign_detector = MultiSignPredictor(holistic, fsize)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Predections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                            ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                            ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                            )     

def display_sentence(frame,sentence):
    cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(frame, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


def dispay_probability(frame,data):
    TEXT_COLOR = (0,0,255)
    
    cv2.putText(frame, "L:"+str(data["L"]), (0, 85+0*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)
    cv2.putText(frame, "R:"+str(data["R"]), (0, 85+1*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)

    cv2.putText(frame, "L2:"+str(data["L2"]), (0, 85+4*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)
    cv2.putText(frame, "R2:"+str(data["R2"]), (0, 85+5*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)


    cv2.putText(frame, "L-D:"+str(data["L-D"]), (0, 85+8*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)
    cv2.putText(frame, "R-D:"+str(data["R-D"]), (0, 85+9*40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_8)


def display_counters(frame,counter,discarded_frames):
    cv2.putText(frame, str(counter), (250, 85+5*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,250,150), 2, cv2.LINE_8)
    cv2.putText(frame, str(discarded_frames), (250, 85+6*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,250,150), 2, cv2.LINE_8)

    


    
    
sentence = []
predictions = []



counter = 0
discarded_frames = 0


cap = cv2.VideoCapture(0)

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()
    if(not ret):
        break
    
    frame,body_keypoints = multi_sign_detector.detect_frame(frame)
    

    if multi_sign_detector.have_sign(frame):
        frame_list = multi_sign_detector.get_frames_indices(frames_no=16)

        for frame_idx in frame_list:
            pytorch_predictor.add_frame(multi_sign_detector[frame_idx])
            keras_predictor.add_frame(multi_sign_detector[frame_idx])

        res1 = pytorch_predictor.predict()
        res2 = keras_predictor.predict()
        res = res1 + res2
        arg_max = np.argmax(res)
        predictions.append(arg_max)
        predictions = predictions[-16:]
        print(predictions)
        multi_sign_detector.truncate_listed_frames()
        sentence.append(actions[arg_max])
        sentence = sentence[-4:]


    image = frame.copy()
    draw_styled_landmarks(image, body_keypoints)
    display_sentence(image,sentence)
    dispay_probability(image,multi_sign_detector.get_data())
    display_counters(image,multi_sign_detector.counter,multi_sign_detector.discarded_frames)


    cv2.imshow("Real-Time", image)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[0]


In [9]:
cap.release()
cv2.destroyAllWindows()

# Create Custom Video

In [1]:
import cv2

cap = cv2.VideoCapture(0)
frames_per_seconds = 15

out = cv2.VideoWriter(f'main2.mp4',0x7634706d, frames_per_seconds, (512,512))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame,(512,512))



    cv2.imshow("Real-Time", frame)
    out.write(frame)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break


out.release()
cap.release()
cv2.destroyAllWindows()
